# construction formule kdb

In [30]:
from qpython import qconnection 
import pandas as pd 
import numpy as np 
from datetime import datetime
import movecolumn as mc

%load_ext autoreload
%autoreload 2

kdb_date = "2022-09-02"
kdb_date = kdb_date.replace('-','.')
print(kdb_date)

## Open a connection to kdb 
q = qconnection.QConnection(host = 'kdb.dts.corp.local',port=8004,username='Administrator',password ='password', pandas = True)

## Retrieves data from KDB
q.open()
kdb_trades=q.sendSync('.engie.getAssessmentsPrices('+kdb_date+')')
q.close() 

## kdb symbols/strings returned as python byte strings -- convert these to regular strings 
bstr_cols = kdb_trades.select_dtypes([object]).columns
for i in bstr_cols:
    kdb_trades[i]=kdb_trades[i].apply(lambda x: x.decode('latin'))

kdb_trades

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022.09.02


,date,ASSESS_DATE,INDEX1,INDEX2,CONTRACT,CONTRACT1_START_DATE,CONTRACT1_END_DATE,CONTRACT2_START_DATE,CONTRACT2_END_DATE,PRODUCT,...,iceeodfutures_physical,trayport_bid,aemopower,iceemission,spectronemission_ask,spectronemission_bid,epex,RISK,VALIDATION_STATUS,COMMENT
0,2022-09-02,2022-09-02,AOC,,2022.9.3.D,2022-09-03,2022-09-03,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotValidated,
1,2022-09-02,2022-09-02,AOC,,2022.35.WE,2022-09-03,2022-09-04,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotValidated,
2,2022-09-02,2022-09-02,AOC,,2022.9.BOM,2022-09-06,2022-09-30,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-50353.658826,NotValidated,
3,2022-09-02,2022-09-02,AOC,,2022.10.M,2022-10-01,2022-10-31,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170374.035463,Validated,Total's price within the threshold -1.3 and 1....
4,2022-09-02,2022-09-02,AOC,,2022.11.M,2022-11-01,2022-11-30,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-33755.818401,Validated,Total's price within the threshold -1.3 and 1....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,2022-09-02,2022-09-02,ZTP,,2028.7.M,2028-07-01,2028-07-31,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotValidated,
12312,2022-09-02,2022-09-02,ZTP,,2028.8.M,2028-08-01,2028-08-31,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotValidated,
12313,2022-09-02,2022-09-02,ZTP,,2028.9.M,2028-09-01,2028-09-30,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotValidated,
12314,2022-09-02,2022-09-02,ZTP,,2028.3.Q,2028-07-01,2028-09-30,NaT,NaT,NG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NotApplicable,


In [31]:
kdb_trades.columns.unique()

Index(['date', 'ASSESS_DATE', 'INDEX1', 'INDEX2', 'CONTRACT',
       'CONTRACT1_START_DATE', 'CONTRACT1_END_DATE', 'CONTRACT2_START_DATE',
       'CONTRACT2_END_DATE', 'PRODUCT', 'PRODUCT_TYPE', 'PROFILE',
       'CONTRACT_TYPE', 'CCY', 'QUANTITY_UNIT', 'RELTAG', 'COMMENT_ID',
       'DTINSERTED', 'LOCATION', 'TYPE', 'REGULATORY_AREA', 'TGP_PRICE',
       'RULE', 'markit accepted_prices', 'markit consensus',
       'markit contributors', 'markit deviation', 'iceeodfutures',
       'iceeodfutures high', 'iceeodfutures low', 'spectroncoal_ask',
       'spectroncoal_bid', 'trayport', 'herengas', 'herengas ask',
       'herengas bid', 'eexpower', 'skylight consensus', 'powernextgasfutures',
       'arguspower', 'arguspower high', 'arguspower low', 'herenpower',
       'herenpower ask', 'herenpower bid', 'herenpower high', 'herenpower low',
       'spectronpower_ask', 'spectronpower_bid', 'cmefutures', 'nasdaqger',
       'icefutures', 'platts', 'prebonlng', 'iceclearedgas', 'spectrongas_as

In [32]:
if "ice" in kdb_trades.columns:
    columns = ['date', 'ASSESS_DATE' , 'INDEX1', 'PROFILE', 'CONTRACT', 'CONTRACT1_START_DATE','TGP_PRICE', 'markit consensus', 'eexpower', 'ice', 'skylight consensus', 'RISK',  'RULE', 'VALIDATION_STATUS', 'COMMENT' ]
    if "skylight consensus" in columns:
            columns = ['date', 'ASSESS_DATE' , 'INDEX1', 'PROFILE', 'CONTRACT', 'CONTRACT1_START_DATE','TGP_PRICE','eexpower', 'ice', 'skylight consensus', 'RISK','RULE', 'VALIDATION_STATUS', 'COMMENT' ]
    else: 
        columns = ['date', 'ASSESS_DATE' , 'INDEX1', 'PROFILE', 'CONTRACT', 'CONTRACT1_START_DATE','TGP_PRICE','eexpower', 'ice', 'RISK', 'RULE',  'VALIDATION_STATUS', 'COMMENT' ]
else: 
    columns = ['date', 'ASSESS_DATE' , 'INDEX1', 'PROFILE', 'CONTRACT', 'CONTRACT1_START_DATE','TGP_PRICE','eexpower', 'RISK',  'VALIDATION_STATUS', 'COMMENT' ]



In [33]:
kdb_trades = kdb_trades.loc[:,columns]
kdb_trades

,date,ASSESS_DATE,INDEX1,PROFILE,CONTRACT,CONTRACT1_START_DATE,TGP_PRICE,RULE,eexpower,RISK,VALIDATION_STATUS,COMMENT
0,2022-09-02,2022-09-02,AOC,BL,2022.9.3.D,2022-09-03,78.000,NoRuleDefined,NaN,NaN,NotValidated,
1,2022-09-02,2022-09-02,AOC,BL,2022.35.WE,2022-09-03,80.000,NoRuleDefined,NaN,NaN,NotValidated,
2,2022-09-02,2022-09-02,AOC,BL,2022.9.BOM,2022-09-06,130.000,NoRuleDefined,NaN,-50353.658826,NotValidated,
3,2022-09-02,2022-09-02,AOC,BL,2022.10.M,2022-10-01,155.665,fixed-1.31.3,NaN,170374.035463,Validated,Total's price within the threshold -1.3 and 1....
4,2022-09-02,2022-09-02,AOC,BL,2022.11.M,2022-11-01,172.729,fixed-1.31.3,NaN,-33755.818401,Validated,Total's price within the threshold -1.3 and 1....
...,...,...,...,...,...,...,...,...,...,...,...,...
12311,2022-09-02,2022-09-02,ZTP,BL,2028.7.M,2028-07-01,36.100,NoRuleDefined,NaN,NaN,NotValidated,
12312,2022-09-02,2022-09-02,ZTP,BL,2028.8.M,2028-08-01,36.443,NoRuleDefined,NaN,NaN,NotValidated,
12313,2022-09-02,2022-09-02,ZTP,BL,2028.9.M,2028-09-01,37.058,NoRuleDefined,NaN,NaN,NotValidated,
12314,2022-09-02,2022-09-02,ZTP,BL,2028.3.Q,2028-07-01,NaN,NoRuleDefined,NaN,NaN,NotApplicable,


In [34]:
kdb_trades.loc[kdb_trades["CONTRACT"].str.contains(".M"),"Contract type"] = "Month"
kdb_trades.loc[kdb_trades["CONTRACT"].str.contains(".Q"),"Contract type"] = "Quarter"
kdb_trades.loc[kdb_trades["CONTRACT"].str.contains(".CAL"),"Contract type"] = "Year"
kdb_trades.dropna(subset = "Contract type", inplace = True)
kdb_trades = mc.MoveToN(kdb_trades,'Contract type',6)

In [35]:
kdb_trades

,date,ASSESS_DATE,INDEX1,PROFILE,CONTRACT,Contract type,CONTRACT1_START_DATE,TGP_PRICE,RULE,eexpower,RISK,VALIDATION_STATUS,COMMENT
2,2022-09-02,2022-09-02,AOC,BL,2022.9.BOM,Month,2022-09-06,130.000000,NoRuleDefined,NaN,-50353.658826,NotValidated,
3,2022-09-02,2022-09-02,AOC,BL,2022.10.M,Month,2022-10-01,155.665000,fixed-1.31.3,NaN,170374.035463,Validated,Total's price within the threshold -1.3 and 1....
4,2022-09-02,2022-09-02,AOC,BL,2022.11.M,Month,2022-11-01,172.729000,fixed-1.31.3,NaN,-33755.818401,Validated,Total's price within the threshold -1.3 and 1....
5,2022-09-02,2022-09-02,AOC,BL,2022.4.Q,Quarter,2022-10-01,170.552598,fixed-1.31.3,NaN,NaN,Validated,"7. MO disagrees, Traders to update curve"
6,2022-09-02,2022-09-02,AOC,BL,2022.12.M,Month,2022-12-01,183.334000,fixed-1.31.3,NaN,-28063.332673,Validated,"7. MO disagrees, Traders to update curve"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,2022-09-02,2022-09-02,ZTP,BL,2028.7.M,Month,2028-07-01,36.100000,NoRuleDefined,NaN,NaN,NotValidated,
12312,2022-09-02,2022-09-02,ZTP,BL,2028.8.M,Month,2028-08-01,36.443000,NoRuleDefined,NaN,NaN,NotValidated,
12313,2022-09-02,2022-09-02,ZTP,BL,2028.9.M,Month,2028-09-01,37.058000,NoRuleDefined,NaN,NaN,NotValidated,
12314,2022-09-02,2022-09-02,ZTP,BL,2028.3.Q,Quarter,2028-07-01,NaN,NoRuleDefined,NaN,NaN,NotApplicable,


In [36]:
uk = kdb_trades.loc[kdb_trades['INDEX1'] == "FRANCE",:]

uk

,date,ASSESS_DATE,INDEX1,PROFILE,CONTRACT,Contract type,CONTRACT1_START_DATE,TGP_PRICE,RULE,eexpower,RISK,VALIDATION_STATUS,COMMENT
2272,2022-09-02,2022-09-02,FRANCE,BL,2022.9.BOM,Month,2022-09-06,550.296758,NoRuleDefined,NaN,NaN,NotValidated,
2273,2022-09-02,2022-09-02,FRANCE,PKS,2022.9.BOM,Month,2022-09-06,759.275000,NoRuleDefined,NaN,NaN,NotValidated,
2274,2022-09-02,2022-09-02,FRANCE,OPKS,2022.9.BOM,Month,2022-09-06,434.197734,NoRuleDefined,NaN,NaN,NotValidated,
2275,2022-09-02,2022-09-02,FRANCE,BL,2022.9.M,Month,2022-09-01,NaN,NoRuleDefined,523.02,NaN,NotApplicable,
2276,2022-09-02,2022-09-02,FRANCE,PKS,2022.9.M,Month,2022-09-01,NaN,NoRuleDefined,636.83,NaN,NotApplicable,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2858,2022-09-02,2022-09-02,FRANCE,PKS,2032.4.Q,Quarter,2032-10-01,148.751170,NoRuleDefined,NaN,NaN,NotValidated,
2859,2022-09-02,2022-09-02,FRANCE,PKS,2032.CAL,Year,2032-01-01,118.920000,NoRuleDefined,NaN,NaN,NotValidated,
2860,2022-09-02,2022-09-02,FRANCE,OPKS,2032.12.M,Month,2032-12-01,103.020149,NoRuleDefined,NaN,NaN,NotValidated,
2861,2022-09-02,2022-09-02,FRANCE,OPKS,2032.4.Q,Quarter,2032-10-01,103.920115,NoRuleDefined,NaN,NaN,NotValidated,


In [37]:
uk['Index'] = uk['INDEX1'] + "_" + uk['PROFILE'] 
uk = mc.MoveToN(uk,'Index',2)
uk

C:\Users\rmolli\AppData\Local\Temp\ipykernel_124776\571209686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk['Index'] = uk['INDEX1'] + "_" + uk['PROFILE']


,date,Index,ASSESS_DATE,INDEX1,PROFILE,CONTRACT,Contract type,CONTRACT1_START_DATE,TGP_PRICE,RULE,eexpower,RISK,VALIDATION_STATUS,COMMENT
2272,2022-09-02,FRANCE_BL,2022-09-02,FRANCE,BL,2022.9.BOM,Month,2022-09-06,550.296758,NoRuleDefined,NaN,NaN,NotValidated,
2273,2022-09-02,FRANCE_PKS,2022-09-02,FRANCE,PKS,2022.9.BOM,Month,2022-09-06,759.275000,NoRuleDefined,NaN,NaN,NotValidated,
2274,2022-09-02,FRANCE_OPKS,2022-09-02,FRANCE,OPKS,2022.9.BOM,Month,2022-09-06,434.197734,NoRuleDefined,NaN,NaN,NotValidated,
2275,2022-09-02,FRANCE_BL,2022-09-02,FRANCE,BL,2022.9.M,Month,2022-09-01,NaN,NoRuleDefined,523.02,NaN,NotApplicable,
2276,2022-09-02,FRANCE_PKS,2022-09-02,FRANCE,PKS,2022.9.M,Month,2022-09-01,NaN,NoRuleDefined,636.83,NaN,NotApplicable,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2858,2022-09-02,FRANCE_PKS,2022-09-02,FRANCE,PKS,2032.4.Q,Quarter,2032-10-01,148.751170,NoRuleDefined,NaN,NaN,NotValidated,
2859,2022-09-02,FRANCE_PKS,2022-09-02,FRANCE,PKS,2032.CAL,Year,2032-01-01,118.920000,NoRuleDefined,NaN,NaN,NotValidated,
2860,2022-09-02,FRANCE_OPKS,2022-09-02,FRANCE,OPKS,2032.12.M,Month,2032-12-01,103.020149,NoRuleDefined,NaN,NaN,NotValidated,
2861,2022-09-02,FRANCE_OPKS,2022-09-02,FRANCE,OPKS,2032.4.Q,Quarter,2032-10-01,103.920115,NoRuleDefined,NaN,NaN,NotValidated,


In [1]:
from kdb import *


In [2]:
from kdb import *
kdb_trades = kdb("2022-09-02", "FRANCE_BL")

mat


In [3]:
kdb_trades

,Date,Index,Contract type,Start Date,TGP,Markit,Markit Std,EEX,Ice,Skylight,Risk,Rule,Validation_status,Comment
2296,2022-09-02,FRANCE_BL,Month,2022-10-01,610.000000,592.885257,7.614936,601.00,595.18,590.992246,2951.611999,fixed-0.10.1,Validated,4. Ok as quarter is ok
2301,2022-09-02,FRANCE_BL,Month,2022-11-01,1180.000000,1141.547180,26.017389,1171.59,1151.87,1146.250046,-27902.714760,fixed-0.10.1,Validated,4. Ok as quarter is ok
2308,2022-09-02,FRANCE_BL,Month,2022-12-01,1141.612903,1191.794802,10.773770,1193.35,1180.99,1194.039536,-170354.763354,fixed-0.10.1,Validated,4. Ok as quarter is ok
2315,2022-09-02,FRANCE_BL,Month,2023-01-01,1095.300529,1118.910793,26.204672,1159.08,1154.90,1122.306811,516574.743169,fixed-0.10.1,Validated,"7. MO disagrees, Traders to update curve"
2318,2022-09-02,FRANCE_BL,Month,2023-02-01,1069.913618,1104.992561,25.957570,1140.45,1136.90,1111.742164,465139.543510,fixed-0.10.1,Validated,"7. MO disagrees, Traders to update curve"
2325,2022-09-02,FRANCE_BL,Month,2023-03-01,783.487171,852.987779,26.245461,845.20,846.25,834.021192,516057.664102,fixed-0.20.2,Validated,"7. MO disagrees, Traders to update curve"
2330,2022-09-02,FRANCE_BL,Month,2023-04-01,396.321875,386.419805,8.411528,NaN,382.91,389.036741,512387.033688,fixed-0.20.2,Validated,"7. MO disagrees, Traders to update curve"
2333,2022-09-02,FRANCE_BL,Month,2023-05-01,335.146736,333.478038,5.115870,NaN,353.05,340.216240,533886.572937,fixed-0.20.2,Validated,ok as price is between markit and ice
2338,2022-09-02,FRANCE_BL,Month,2023-06-01,356.965398,354.379658,3.869608,NaN,349.97,352.128435,533682.068548,fixed-0.20.2,Validated,"7. MO disagrees, Traders to update curve"
2343,2022-09-02,FRANCE_BL,Month,2023-07-01,322.350522,330.067542,4.934433,NaN,321.04,331.242817,442637.695601,fixed-0.20.2,Validated,ok as price is between markit and ice


In [4]:
kdb_plot(kdb_trades,"Month", Ice = True, Skylight = True)

In [5]:
    from qpython import qconnection 
    import pandas as pd 
    import numpy as np 
    from datetime import datetime
    import movecolumn as mc

    print ("mawarae")
    kdb_date = "2022-08-31"
    kdb_date = kdb_date.replace('-','.')
 
    ## Open a connection to kdb 
    q = qconnection.QConnection(host = 'kdb.dts.corp.local',port = 8004,username = 'Administrator',password = 'password', pandas = True)

    ## Retrieves data from KDB
    q.open()
    kdb_trades=q.sendSync('.engie.getAssessmentsPrices('+kdb_date+')')
    q.close()

mawarae


In [7]:
kdb_trades.columns

Index(['date', 'ASSESS_DATE', 'INDEX1', 'INDEX2', 'CONTRACT',
       'CONTRACT1_START_DATE', 'CONTRACT1_END_DATE', 'CONTRACT2_START_DATE',
       'CONTRACT2_END_DATE', 'PRODUCT', 'PRODUCT_TYPE', 'PROFILE',
       'CONTRACT_TYPE', 'CCY', 'QUANTITY_UNIT', 'RELTAG', 'COMMENT_ID',
       'DTINSERTED', 'LOCATION', 'TYPE', 'REGULATORY_AREA', 'TGP_PRICE',
       'RULE', 'markit accepted_prices', 'markit consensus',
       'markit contributors', 'markit deviation', 'iceeodfutures',
       'iceeodfutures high', 'iceeodfutures low', 'eexpower',
       'skylight consensus', 'powernextgasfutures', 'herenpower',
       'herenpower ask', 'herenpower bid', 'herenpower high', 'herenpower low',
       'cmefutures', 'trayport', 'nasdaqger', 'icefutures', 'platts',
       'prebonlng', 'herengas', 'herengas ask', 'herengas bid',
       'iceclearedgas', 'spectrongas_ask', 'spectrongas_bid', 'gccoal_ask',
       'gccoal_bid', 'nasdaqomx', 'nasdaqomx ask', 'nasdaqomx bid', 'omipelec',
       'ngxclearedgasus